In [1]:
from datetime import datetime, timedelta, timezone
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import pyautogui
import time

pd.options.display.float_format = '{:,.2f}'.format

# Download data

### Fetching tickers from nasdaq and filtering out ETFs and stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason

    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

In [3]:
def get_data(tickers_list, p="max"):
    return yf.download(tickers_list, period=p, interval="1d")

In [104]:
tickers = get_all_tickers()[:100]
# tickers = ["TSLA", "AAPL", "MSFT", "NVDA", "GOOG"]
data = get_data(tickers)

[*********************100%%**********************]  100 of 100 completed


In [105]:
data_cop = data.copy()

In [106]:
new_df = pd.DataFrame(index=data_cop.index.year.unique(), columns=data_cop.columns)
new_df.drop("Adj Close", axis=1, level=0, inplace=True)
new_df["Volume"] = data_cop["Volume"].groupby(data_cop.index.year).mean()
new_df["Close"] = data_cop["Close"].groupby(data_cop.index.year).last()
new_df["Open"] = data_cop["Open"].groupby(data_cop.index.year).first()
new_df["High"] = data_cop["High"].groupby(data_cop.index.year).max()
new_df["Low"] = data_cop["Low"].groupby(data_cop.index.year).min()
for stock in new_df["Close"].columns:
    new_df["Change", stock] = np.nan
new_df["Change"] = new_df.xs("Close", axis=1, level=0).pct_change() * 100

C:\Users\sasso\AppData\Local\Temp\ipykernel_5824\4084708284.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df["Change", stock] = np.nan


In [107]:
stocks_checklist = pd.DataFrame(index=["Years up", "Div yield", "Sector", "avg daily vol", "Total Revenue Incr", "Gross Margin", "Gross Profit Incr"], columns=new_df["Close"].columns)
stocks_checklist

,A,AA,AACG,AACI,AACIU,AACIW,AACT,AADI,AAGR,AAGRW,...,ACRV,ACST,ACT,ACTG,ACU,ACV,ACVA,ACXP,ADAG,ADAP
Years up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Div yield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
avg daily vol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue Incr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Margin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit Incr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
tickers = list(new_df["Close"].columns)  # In case some of the tickers failed, this is the ticker list straight from the returned df
# Years up in a row
tmp = new_df["Change"].iloc[::-1]
neg_year = tmp.apply(lambda x: x[x < 0].index[0] if any(x<0) else None)
current_year = tmp.index[0]
stocks_checklist.loc["Years up"] = current_year - neg_year
stocks_checklist.loc["avg daily vol"] = new_df["Volume"].iloc[-1:].mean()
# Div yield
print(f"0/{len(tickers)}")
i = 0
for ticker in tickers:
    clear_output(wait=True)
    print(ticker)
    print(f"{i}/{len(tickers)}")
    i += 1
    # if i < 25:
        # continue
    tmp = yf.Ticker(ticker)
    start_date = (datetime.now() - timedelta(days = 365))
    start_date = start_date.replace(tzinfo=timezone.utc)
    div_sum = tmp.dividends.loc[start_date:].sum()
    close_price = new_df["Close"][ticker].iloc[-1]
    div_yield = (div_sum/close_price) * 100
    stocks_checklist.loc["Div yield"][ticker] = div_yield

    # Sector
    if "sector" in tmp.info:
        stocks_checklist.loc["Sector"][ticker] = tmp.info["sector"]
    
    if "Gross Profit" in tmp.income_stmt.columns:
        # Average Gross margin over the last 3 income statements
        stocks_checklist.loc["Gross Margin"][ticker] = ((tmp.income_stmt.loc["Gross Profit"].iloc[::-1] / tmp.income_stmt.loc["Total Revenue"].iloc[::-1]) * 100).mean()  # Gross margin of the last 3 years
        
        # Gross profit increase percentage over the last 3 years (3 income statements)
        stocks_checklist.loc["Gross Profit Incr"][ticker] = tmp.income_stmt.loc["Gross Profit"].iloc[::-1].pct_change().mean() * 100
    
    # Total Revenue increase percentage over the last 3 years (3 income statements)
    stocks_checklist.loc["Total Revenue Incr"][ticker] = tmp.income_stmt.loc["Total Revenue"].iloc[::-1].pct_change().mean() * 100
    
stocks_checklist.loc["Div yield"].replace(0.0, np.nan, inplace=True)

ADAP
99/100


In [138]:
stocks_checklist

,A,AA,AACG,AACI,AACIU,AACIW,AACT,AADI,AAGR,AAGRW,...,ACRV,ACST,ACT,ACTG,ACU,ACV,ACVA,ACXP,ADAG,ADAP
Years up,0.00,0.00,0.00,NaN,NaN,NaN,NaN,0.00,0.00,NaN,...,0.00,0.00,0.00,0.00,2.00,2.00,0.00,1.00,2.00,1.00
Div yield,0.70,1.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.72,NaN,1.32,10.94,NaN,NaN,NaN,NaN
Sector,Healthcare,Basic Materials,Consumer Defensive,Financial Services,Financial Services,Financial Services,Financial Services,Healthcare,Consumer Defensive,Consumer Defensive,...,Healthcare,Healthcare,Financial Services,Industrials,Consumer Defensive,Financial Services,Consumer Cyclical,Healthcare,Healthcare,Healthcare
avg daily vol,"1,641,677.78","5,312,355.56","14,144.44","27,700.00",0.00,0.00,"49,822.22","484,377.78","447,844.44",477.00,...,"76,811.11","23,088.89","314,333.33","117,866.67","20,088.89","40,566.67","1,165,477.78","192,622.22","134,300.00","907,155.56"
Total Revenue Incr,8.84,16.66,13.49,NaN,NaN,NaN,NaN,664.37,inf,inf,...,NaN,inf,-0.51,81.45,8.77,"-1,036.61",44.82,NaN,671.52,198.43
Gross Margin,53.03,18.95,46.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gross Profit Incr,7.48,51.19,31.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Year up - how many years the stock is uptrending in a row
for ticker in tickers:
    new_df["Change"][ticker]

In [11]:
tickers_stats = yf.Tickers(tickers)

In [12]:
for ticker in tickers_stats.tickers:
    print(tickers_stats.tickers[ticker].info)
    break

{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add

In [13]:
tmp = new_df["Change"]["TSLA"].iloc[::-1]
tmp.index[0]

2024

In [14]:
tmp < 0

Date
2024     True
2023    False
2022     True
2021    False
2020    False
2019    False
2018    False
2017    False
2016     True
2015    False
2014    False
2013    False
2012    False
2011    False
2010    False
2009    False
2008    False
2007    False
2006    False
2005    False
2004    False
2003    False
2002    False
2001    False
2000    False
1999    False
1998    False
1997    False
1996    False
1995    False
1994    False
1993    False
1992    False
1991    False
1990    False
1989    False
1988    False
1987    False
1986    False
1985    False
1984    False
1983    False
1982    False
1981    False
1980    False
Name: TSLA, dtype: bool

In [34]:
new_df["Volume"].iloc[-1:].mean()

AAPL    57,060,266.67
GOOG    17,562,877.78
MSFT    23,245,033.33
NVDA    48,354,411.11
TSLA   102,556,544.44
dtype: float64

In [130]:
msft = yf.Ticker("AACI")

In [110]:
pd.set_option('display.max_rows', None)
msft.balance_sheet
# pd.reset_option('display.max_rows')

In [97]:
((msft.income_stmt.loc["Gross Profit"].iloc[::-1] / msft.income_stmt.loc["Total Revenue"].iloc[::-1]) * 100).mean()  # Gross margin of the last 3 years

68.5071407829007

In [131]:
msft.info

{'address1': '1760 Market Street',
 'address2': 'Suite 602',
 'city': 'Philadelphia',
 'state': 'PA',
 'zip': '19103',
 'country': 'United States',
 'phone': '215 543 6886',
 'website': 'https://armadaacq.com',
 'industry': 'Shell Companies',
 'industryKey': 'shell-companies',
 'industryDisp': 'Shell Companies',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'Armada Acquisition Corp. I does not have significant operations. It intends to effect a merger, capital stock exchange, asset acquisition, stock purchase, reorganization, or related business combination with one or more businesses in the financial technology industry. The company was incorporated in 2020 and is based in Philadelphia, Pennsylvania.',
 'companyOfficers': [{'maxAge': 1,
   'name': 'Mr. Stephen P. Herbert',
   'age': 61,
   'title': 'CEO & Chairman',
   'yearBorn': 1962,
   'exercisedValue': 0,
   'unexercisedValue': 0},
  {'maxAge': 1,

In [133]:
new_df["Change"]["AACI"].tail(20)

Date
2005    NaN
2006    NaN
2007    NaN
2008    NaN
2009    NaN
2010    NaN
2011    NaN
2012    NaN
2013    NaN
2014    NaN
2015    NaN
2016    NaN
2017    NaN
2018    NaN
2019    NaN
2020    NaN
2021    NaN
2022   3.37
2023   6.42
2024   1.21
Name: AACI, dtype: float64